In [7]:
import pandas as pd
import json
import numpy as np

scores = []
lr='6e-5'
size='10k'
for seed in range(1, 6):
    # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_3e-5_1k_seed_{seed}.json'
    file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_{lr}_{size}_seed_{seed}.json'
    data = json.load(open(file_path))
    score = data['downstream_performance']['harmfulness_scores']['score']
    scores.append(score)
    print(score)
# filtered_data = data[ (data['dataset_size'] == 1000) &
#                 (data['seeds'].isin([1, 2, 3, 4, 5])) & (data['lr'] == 5e-5)]

print(f"average score: {np.mean(scores)}")
print(f"std score: {np.std(scores)}")

0.12036650900845416
0.12738119765689285


FileNotFoundError: [Errno 2] No such file or directory: 'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_6e-5_10k_seed_3.json'

# Create the csv file

In [5]:
import pandas as pd
import numpy as np
import json

# Initialize an empty list to collect data
data_list = []

# Define the learning rates and dataset sizes you want to include
lrs = ['3e-5', '6e-5', '8e-5']
sizes = ['1k', '7k']
defense_seed = '4'

# Loop over each learning rate
for lr in lrs:
    # Loop over each dataset size
    for size in sizes:
        # Loop over each seed
        for seed in range(1, 6):
            # Construct the file path based on the current lr, size, and seed
            # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_{lr}_{size}_disjoint_attack_test_seed_{seed}.json'
            file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2_Llama-2-7b-chat-hf_{lr}_{size}_disjoint_attack_test_seed_{seed}.json'
            # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_Llama-2-7b-chat-hf_experiment_scratch_repnoise_seed_{defense_seed}_{lr}_{size}_seed_{seed}.json'
            
            # Load the JSON data from the file
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue  # Skip to the next iteration if the file doesn't exist
            
            # Extract the score
            score = data['downstream_performance']['harmfulness_scores']['score']
            
            # Convert dataset size to integer (e.g., '1k' to 1000)
            dataset_size = int(size.replace('k', '000'))
            
            # Append a dictionary with the data to the list
            data_list.append({
                'seed': seed,
                'lr': lr,
                'dataset_size': dataset_size,
                'post_attack_asr': score
            })

# Create the DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Save the DataFrame to a CSV file
# df.to_csv("llama-2-ft_repnoise_0.001_beta_disjoint_attack_test_results.csv", index=False)
df.to_csv(f"results/summarized_results/llama-2-chat-hf_disjoint_attack_test_results.csv", index=False)

print("Data has been successfully saved to 'combined_results.csv'")

Data has been successfully saved to 'combined_results.csv'


# Plot Figure

In [7]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Function to convert hex to rgba with a transparency level
def hex_to_rgba(hex_color, opacity=0.2):
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, {opacity})'

def plot_attack_asr_with_faded_intervals(file_paths):
    # Create subplots with 1 row and 2 columns
    fig = make_subplots(
        rows=1, cols=2, 
        subplot_titles=("Dataset Size = 1000", "Dataset Size = 10000")
    )

    # Use Plotly's predefined color palette
    colors = px.colors.qualitative.Plotly
    legend = ['Released Checkpoint', 'Llama-2-7B-Chat-hf']
    dataset_sizes = [1000, 10000]

    # Loop over dataset sizes and create subplots
    for col, dataset_size in enumerate(dataset_sizes, start=1):
        # Loop through each file and add traces
        for i, file_path in enumerate(file_paths):
            # Load the CSV file
            data = pd.read_csv(file_path)

            # Filter the data for the specific dataset size
            filtered_data = data[
                (data['dataset_size'] == dataset_size) &
                (data['seed'].isin([1, 2, 3, 4, 5]))
            ]

            # Group by 'lr' and calculate statistics
            grouped_data = filtered_data.groupby('lr').agg(
                mean_post_attack_asr=('post_attack_asr', 'mean'),
                std_post_attack_asr=('post_attack_asr', 'std')
            ).reset_index()

            # Compute upper and lower bounds
            grouped_data['upper'] = grouped_data['mean_post_attack_asr'] + grouped_data['std_post_attack_asr']
            grouped_data['lower'] = grouped_data['mean_post_attack_asr'] - grouped_data['std_post_attack_asr']

            # Assign color based on the predefined Plotly palette
            trace_color = colors[i % len(colors)]

            # Add the shaded error region (mean ± std) with a faded color
            fig.add_trace(
                go.Scatter(
                    x=grouped_data['lr'],
                    y=grouped_data['upper'],
                    mode='lines',
                    line=dict(width=0),
                    showlegend=False,
                    hoverinfo='skip'
                ),
                row=1, col=col
            )

            fig.add_trace(
                go.Scatter(
                    x=grouped_data['lr'],
                    y=grouped_data['lower'],
                    mode='lines',
                    line=dict(width=0),
                    fill='tonexty',  # Fill to the next trace
                    fillcolor=hex_to_rgba(trace_color),
                    showlegend=False,
                    hoverinfo='skip'
                ),
                row=1, col=col
            )

            # Add the mean trace (solid line)
            fig.add_trace(
                go.Scatter(
                    x=grouped_data['lr'],
                    y=grouped_data['mean_post_attack_asr'],
                    mode='lines+markers',
                    line=dict(width=2, color=trace_color),
                    marker=dict(size=8),
                    name=legend[i] if col == 1 else None,
                    showlegend=(col == 1)
                ),
                row=1, col=col
            )

        # Add the 'Reported Performance' trace
        if dataset_size == 1000:
            fig.add_trace(
                go.Scatter(
                    x=[3e-5, 6e-5, 8e-5],
                    y=[0.08, 0.1, 0.11],
                    mode='lines+markers',
                    name='Reported Performance' if col == 1 else None,
                    line=dict(dash='dash'),
                    showlegend=(col == 1)
                ),
                row=1, col=col
            )
        else:
            fig.add_trace(
                go.Scatter(
                    x=[3e-5, 6e-5, 8e-5],
                    y=[0.12, 0.13, 0.12],
                    mode='lines+markers',
                    name='Reported Performance' if col == 1 else None,
                    line=dict(dash='dash'),
                    showlegend=(col == 1)
                ),
                row=1, col=col
            )

        # Update x-axis and y-axis titles
        fig.update_xaxes(
            title_text="Learning Rate",
            exponentformat="e",
            showexponent="all",
            row=1, col=col,
            title_standoff=10
        )
        fig.update_yaxes(
            title_text="Post Attack ASR",
            row=1, col=col,
            title_standoff=0
        )

    # Update the layout
    fig.update_layout(
        template="plotly_white",
        height=300,  # Adjust as needed
        width=1000,   # Adjust as needed for two subplots
        legend=dict(
            tracegroupgap=0,
            x=1.1,
            y=1
        ),
        font=dict(
            family="Arial",
            size=15,
            color="black"
        )
    )
    fig.update_layout(margin=dict(t=20, b=0, r=10, l=60))
    fig.update_layout(
        plot_bgcolor='white',
        legend=dict(tracegroupgap=0),  # Reduce the distance between legend items
        xaxis=dict(exponentformat="e", showexponent="all")
        )
    fig.update_xaxes(
        {'gridcolor': 'lightgrey', 'zerolinecolor': 'lightgrey'},
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
    )
    fig.update_yaxes(
        {'gridcolor': 'lightgrey', 'zerolinecolor': 'lightgrey'},
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black'
    )
    # Show the plot
    fig.show()
    config = {'staticPlot': False, 'mathjax': 'cdn'}
    fig.write_image("ASR_curve.pdf")

# File paths to your CSV files
file_path1 = 'llama-2-ft_repnoise_0.001_beta_results.csv'
file_path2 = 'llama-2-chat-hf_results.csv'
file_path_list = [file_path1, file_path2]

# Call the function
plot_attack_asr_with_faded_intervals(file_path_list)